In [1]:
import pandas as pd
import numpy as np

In [2]:
TEAMS = {"MIBR": 0, 
         "Leviatan": 1, 
         "Sentinels": 2, 
         "NRG Esports": 3, 
         "FURIA": 4, 
         "100 Thieves": 5, 
         "LOUD" : 6, 
         "Evil Geniuses" : 7, 
         "G2 Esports" : 8, 
         "Cloud9" : 9, 
         "KRÃœ Esports" : 10, 
         "Titan Esports Club" : 11, 
         "JDG Esports" : 12, 
         "All Gamers" : 13, 
         "TYLOO" : 14, 
         "Bilibili Gaming" : 15, 
         "Wolves Esports" : 16, 
         "Dragon Ranger Gaming" : 17, 
         "Nova Esports" : 18, 
         "FunPlus Phoenix" : 19, 
         "Trace Esports" : 20, 
         "EDward Gaming" : 21, 
         "ZETA DIVISION" : 22, 
         "DRX" : 23, 
         "Team Secret" : 24, 
         "BLEED" : 25, 
         "T1" : 26, 
         "Gen.G" : 27, 
         "Paper Rex" : 28, 
         "Talon Esports" : 29, 
         "Rex Regum Qeon" : 30, 
         "DetonatioN FocusMe" : 31, 
         "Global Esports" : 32, 
         "FUT Esports" : 33, 
         "KOI" : 34, 
         "BBL Esports" : 35, 
         "FNATIC" : 36, 
         "Team Heretics" : 37, 
         "Natus Vincere" : 38, 
         "GIANTX" : 39, 
         "Gentle Mates" : 40, 
         "Team Vitality" : 41, 
         "Team Liquid" : 42, 
         "Karmine Corp": 43}

MAPS = {"Bind" : 0, 
        "Haven" : 1, 
        "Split" : 2, 
        "Ascent" : 3,
        "Icebox" : 4, 
        "Breeze" : 5, 
        "Fracture" : 6, 
        "Abyss" : 7, 
        "Lotus" : 8, 
        "Sunset" : 9, 
        "Pearl" : 10}

In [20]:
data = pd.read_csv("Dataset/maps_scores.csv")
data = data[["Team A", "Team B", "Team A Score", "Team B Score"]]
data["Winner"] = data["Team A Score"] - data["Team B Score"]
data

,Team A,Team B,Team A Score,Team B Score,Winner
0,MIBR,Leviatán,9,13,-4
1,MIBR,Leviatán,7,13,-6
2,Sentinels,NRG Esports,13,8,5
3,Sentinels,NRG Esports,14,12,2
4,FURIA,100 Thieves,14,12,2
...,...,...,...,...,...
1013,DRX,Gen.G,8,13,-5
1014,Paper Rex,Gen.G,5,13,-8
1015,Paper Rex,Gen.G,13,4,9
1016,Paper Rex,Gen.G,11,13,-2


In [24]:
results = data[["Team A", "Team B", "Winner"]]
results["Team A"].replace(TEAMS, inplace=True)
results["Team B"].replace(TEAMS, inplace=True)

winners = pd.read_csv("Dataset/scores.csv")

C:\Users\lelin\AppData\Local\Temp\ipykernel_14828\1534551706.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  results["Team A"].replace(TEAMS, inplace=True)
C:\Users\lelin\AppData\Local\Temp\ipykernel_14828\1534551706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results["Team A"].replace(TEAMS, inplace=True)
C:\Users\lelin\AppData\Lo

,Team A,Team B,Winner
0,0,Leviatán,-4
1,0,Leviatán,-6
2,2,3,5
3,2,3,2
4,4,5,2
...,...,...,...
1013,23,27,-5
1014,28,27,-8
1015,28,27,9
1016,28,27,-2
